In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
# Upload your CSV from local
from google.colab import files
uploaded = files.upload()

# Load it into a DataFrame
df_original = pd.read_csv("sva.csv")  # replace with actual filename
df_original.head()



Saving sva.csv to sva (1).csv


,result_id,result_code,combination_type,customer_name,question_id,question_text,end_date,survey_id,survey_year,answer_value,combination_value,usergroup
0,116234,426R,Promoter_Passive_Detractor,Swiss Life Select Österreich,883,"Wie wahrscheinlich ist es, dass Sie Swiss Life...",9999-12-31 23:59:59.999999,406,2025,2,Promoter,Privatkunde / B2C
1,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,647,"Der Service, den ich erlebe, passt zum Markenv...",9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
2,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,668,"hat es mir leicht gemacht, mein Anliegen zu lö...",9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
3,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,682,Ich fühle mich vom Unternehmen stets fair beha...,9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
4,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,827,den Produkten und/oder Dienstleistungen?,9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C


In [ ]:
df=df_original

In [ ]:
# Step 1: Get only result_ids that have question 883
valid_result_ids = df[df["question_id"] == 883]["result_id"].unique()

# Step 2: Filter the full DataFrame to only keep those result_ids (left join style)
df = df[df["result_id"].isin(valid_result_ids)]


In [ ]:
# List of other question_ids
other_questions = [668, 682, 827, 647]

# Filter logic
df = df[~(
    ((df["question_id"] == 883) & (df["answer_value"] == 12)) |
    ((df["question_id"].isin(other_questions)) & (df["answer_value"] == 6))
)]


In [ ]:
df.head()

,result_id,result_code,combination_type,customer_name,question_id,question_text,end_date,survey_id,survey_year,answer_value,combination_value,usergroup
0,116234,426R,Promoter_Passive_Detractor,Swiss Life Select Österreich,883,"Wie wahrscheinlich ist es, dass Sie Swiss Life...",9999-12-31 23:59:59.999999,406,2025,2,Promoter,Privatkunde / B2C
1,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,647,"Der Service, den ich erlebe, passt zum Markenv...",9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
2,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,668,"hat es mir leicht gemacht, mein Anliegen zu lö...",9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
3,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,682,Ich fühle mich vom Unternehmen stets fair beha...,9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C
4,116234,426R,Top_Middle_Flop,Swiss Life Select Österreich,827,den Produkten und/oder Dienstleistungen?,9999-12-31 23:59:59.999999,406,2025,1,Top2,Privatkunde / B2C


In [ ]:
# Pivot question_id = 883
df_883 = df[df["question_id"] == 883].pivot_table(
    index=["result_id", "customer_name", "usergroup"],
    values=["answer_value", "combination_value"],
    aggfunc="first"
).reset_index().rename(columns={
    "answer_value": "answer_q883",
    "combination_value": "combination_q883"
})

# List of other question_ids to compare
other_questions = [668, 682, 827, 647]

# Dictionary to store final merged DataFrames
final_dfs = {}

# Loop through each question and merge with 883
for qid in other_questions:
    df_other = df[df["question_id"] == qid].pivot_table(
        index=["result_id", "customer_name", "usergroup"],
        values=["answer_value", "combination_value"],
        aggfunc="first"
    ).reset_index().rename(columns={
        "answer_value": f"answer_q{qid}",
        "combination_value": f"combination_q{qid}"
    })

    # Merge with 883 data
    final_df = pd.merge(df_883, df_other, on=["result_id", "customer_name", "usergroup"], how="inner")

    # Store in dictionary
    final_dfs[qid] = final_df

# Access each merged DataFrame
final_df_668 = final_dfs[668]
final_df_682 = final_dfs[682]
final_df_827 = final_dfs[827]
final_df_647 = final_dfs[647]

# Optional: Show one result
print("Preview of final_df_682:")
print(final_df_682.head())


Preview of final_df_682:
   result_id                 customer_name          usergroup  answer_q883  \
0     116234  Swiss Life Select Österreich  Privatkunde / B2C            2   
1     116235  Swiss Life Select Österreich  Privatkunde / B2C            1   
2     116236  Swiss Life Select Österreich  Privatkunde / B2C            1   
3     116238  Swiss Life Select Österreich  Privatkunde / B2C            1   
4     116239  Swiss Life Select Österreich  Privatkunde / B2C            1   

  combination_q883  answer_q682 combination_q682  
0         Promoter            1             Top2  
1         Promoter            1             Top2  
2         Promoter            1             Top2  
3         Promoter            1             Top2  
4         Promoter            1             Top2  


In [ ]:
customer_names = final_df_668['customer_name'].unique()
summary_data = []

for k in customer_names:
    customer_df = final_df_668[final_df_668['customer_name'] == k]

    if k == 'xy':
        user_groups = customer_df['usergroup'].unique()
        for m in user_groups:
            user_df = customer_df[customer_df['usergroup'] == m]
            for i in range(1, 6):
                df_top = user_df[user_df['answer_q668'] == i].copy()
                df_top['NPS_Category'] = df_top['combination_q883']
                promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
                detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
                total = len(df_top)
                nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
                summary_data.append({'Customer Name': k, 'User Group': m, 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
            df_all = user_df.copy()
            df_all['NPS_Category'] = df_all['combination_q883']
            promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
            detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
            total = len(df_all)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data.append({'Customer Name': k, 'User Group': m, 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
    else:
        for i in range(1, 6):
            df_top = customer_df[customer_df['answer_q668'] == i].copy()
            df_top['NPS_Category'] = df_top['combination_q883']
            promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
            detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
            total = len(df_top)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data.append({'Customer Name': k, 'User Group': 'All', 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
        df_all = customer_df.copy()
        df_all['NPS_Category'] = df_all['combination_q883']
        promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
        detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
        total = len(df_all)
        nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
        summary_data.append({'Customer Name': k, 'User Group': 'All', 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})

summary_df_668 = pd.DataFrame(summary_data)
# Total all responses across customers, using only the 'Insgesamt' rows
total_all_responses = summary_df_668[summary_df_668['Value'] == 'Insgesamt']['Total Responses'].sum()
# Calculate weight_base using the number of unique customers
unique_customers = summary_df_668['Customer Name'].nunique()
weight_base = total_all_responses / unique_customers
# Add Weight_Base to the DataFrame
summary_df_668['Weight_Base'] = weight_base
customer_totals = summary_df_668[summary_df_668['Value'] == 'Insgesamt'].set_index('Customer Name')['Total Responses'].to_dict()
summary_df_668['Weight_Company'] = summary_df_668['Customer Name'].map(customer_totals).apply(lambda x: weight_base / x if x > 0 else None)
summary_df_668['Weighted_Promoters'] = summary_df_668['Promoters'] * summary_df_668['Weight_Company']
summary_df_668['Weighted_Detractors'] = summary_df_668['Detractors'] * summary_df_668['Weight_Company']


In [ ]:
customer_names = final_df_682['customer_name'].unique()
summary_data_682 = []

for k in customer_names:
    customer_df = final_df_682[final_df_682['customer_name'] == k]
    if k == 'xy':
        user_groups = customer_df['usergroup'].unique()
        for m in user_groups:
            user_df = customer_df[customer_df['usergroup'] == m]
            for i in range(1, 6):
                df_top = user_df[user_df['answer_q682'] == i].copy()
                df_top['NPS_Category'] = df_top['combination_q883']
                promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
                detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
                total = len(df_top)
                nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
                summary_data_682.append({'Customer Name': k, 'User Group': m, 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
            df_all = user_df.copy()
            df_all['NPS_Category'] = df_all['combination_q883']
            promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
            detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
            total = len(df_all)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_682.append({'Customer Name': k, 'User Group': m, 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
    else:
        for i in range(1, 6):
            df_top = customer_df[customer_df['answer_q682'] == i].copy()
            df_top['NPS_Category'] = df_top['combination_q883']
            promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
            detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
            total = len(df_top)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_682.append({'Customer Name': k, 'User Group': 'All', 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})
        df_all = customer_df.copy()
        df_all['NPS_Category'] = df_all['combination_q883']
        promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
        detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
        total = len(df_all)
        nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
        summary_data_682.append({'Customer Name': k, 'User Group': 'All', 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors})

summary_df_682 = pd.DataFrame(summary_data_682)
# Total all responses across customers, using only the 'Insgesamt' rows
total_all_responses = summary_df_682[summary_df_682['Value'] == 'Insgesamt']['Total Responses'].sum()
# Calculate weight_base using the number of unique customers
unique_customers = summary_df_682['Customer Name'].nunique()
weight_base = total_all_responses / unique_customers
# Add Weight_Base to the DataFrame
summary_df_668['Weight_Base'] = weight_base
customer_totals = summary_df_682[summary_df_682['Value'] == 'Insgesamt'].set_index('Customer Name')['Total Responses'].to_dict()
summary_df_682['Weight_Company'] = summary_df_682['Customer Name'].map(customer_totals).apply(lambda x: weight_base / x if x > 0 else None)
summary_df_682['Weighted_Promoters'] = summary_df_682['Promoters'] * summary_df_682['Weight_Company']
summary_df_682['Weighted_Detractors'] = summary_df_682['Detractors'] * summary_df_682['Weight_Company']


In [ ]:
customer_names = final_df_827['customer_name'].unique()
summary_data_827 = []

for k in customer_names:
    customer_df = final_df_827[final_df_827['customer_name'] == k]
    if k == 'xy':
        user_groups = customer_df['usergroup'].unique()
        for m in user_groups:
            user_df = customer_df[customer_df['usergroup'] == m]
            for i in range(1, 6):
                df_top = user_df[user_df['answer_q827'] == i].copy()
                df_top['NPS_Category'] = df_top['combination_q883']
                promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
                detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
                total = len(df_top)
                nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
                summary_data_827.append({'Customer Name': k, 'User Group': m, 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
            df_all = user_df.copy()
            df_all['NPS_Category'] = df_all['combination_q883']
            promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
            detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
            total = len(df_all)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_827.append({'Customer Name': k, 'User Group': m, 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
    else:
        for i in range(1, 6):
            df_top = customer_df[customer_df['answer_q827'] == i].copy()
            df_top['NPS_Category'] = df_top['combination_q883']
            promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
            detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
            total = len(df_top)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_827.append({'Customer Name': k, 'User Group': 'All', 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
        df_all = customer_df.copy()
        df_all['NPS_Category'] = df_all['combination_q883']
        promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
        detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
        total = len(df_all)
        nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
        summary_data_827.append({'Customer Name': k, 'User Group': 'All', 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})

summary_df_827 = pd.DataFrame(summary_data_827)
# Total all responses across customers, using only the 'Insgesamt' rows
total_all_responses = summary_df_827[summary_df_827['Value'] == 'Insgesamt']['Total Responses'].sum()
# Calculate weight_base using the number of unique customers
unique_customers = summary_df_827['Customer Name'].nunique()
weight_base = total_all_responses / unique_customers
# Add Weight_Base to the DataFrame
summary_df_668['Weight_Base'] = weight_base
customer_totals = summary_df_827[summary_df_827['Value'] == 'Insgesamt'].set_index('Customer Name')['Total Responses'].to_dict()
summary_df_827['Weight_Company'] = summary_df_827['Customer Name'].map(customer_totals).apply(lambda x: weight_base / x if x > 0 else None)
summary_df_827['Weighted_Promoters'] = summary_df_827['Promoters'] * summary_df_827['Weight_Company']
summary_df_827['Weighted_Detractors'] = summary_df_827['Detractors'] * summary_df_827['Weight_Company']


In [ ]:
customer_names = final_df_647['customer_name'].unique()
summary_data_647 = []

for k in customer_names:
    customer_df = final_df_647[final_df_647['customer_name'] == k]
    if k == 'xy':
        user_groups = customer_df['usergroup'].unique()
        for m in user_groups:
            user_df = customer_df[customer_df['usergroup'] == m]
            for i in range(1, 6):
                df_top = user_df[user_df['answer_q647'] == i].copy()
                df_top['NPS_Category'] = df_top['combination_q883']
                promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
                detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
                total = len(df_top)
                nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
                summary_data_647.append({'Customer Name': k, 'User Group': m, 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
            df_all = user_df.copy()
            df_all['NPS_Category'] = df_all['combination_q883']
            promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
            detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
            total = len(df_all)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_647.append({'Customer Name': k, 'User Group': m, 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
    else:
        for i in range(1, 6):
            df_top = customer_df[customer_df['answer_q647'] == i].copy()
            df_top['NPS_Category'] = df_top['combination_q883']
            promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
            detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
            total = len(df_top)
            nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
            summary_data_647.append({'Customer Name': k, 'User Group': 'All', 'Value': i, 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})
        df_all = customer_df.copy()
        df_all['NPS_Category'] = df_all['combination_q883']
        promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
        detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
        total = len(df_all)
        nps_score = ((promoters - detractors) / total) * 100 if total > 0 else None
        summary_data_647.append({'Customer Name': k, 'User Group': 'All', 'Value': 'Insgesamt', 'Total Responses': total, 'Promoters': promoters, 'Detractors': detractors, 'NPS Score': nps_score})

summary_df_647 = pd.DataFrame(summary_data_647)
# Total all responses across customers, using only the 'Insgesamt' rows
total_all_responses = summary_df_647[summary_df_647['Value'] == 'Insgesamt']['Total Responses'].sum()
# Calculate weight_base using the number of unique customers
unique_customers = summary_df_647['Customer Name'].nunique()
weight_base = total_all_responses / unique_customers
# Add Weight_Base to the DataFrame
summary_df_668['Weight_Base'] = weight_base
customer_totals = summary_df_647[summary_df_647['Value'] == 'Insgesamt'].set_index('Customer Name')['Total Responses'].to_dict()
summary_df_647['Weight_Company'] = summary_df_647['Customer Name'].map(customer_totals).apply(lambda x: weight_base / x if x > 0 else None)
summary_df_647['Weighted_Promoters'] = summary_df_647['Promoters'] * summary_df_647['Weight_Company']
summary_df_647['Weighted_Detractors'] = summary_df_647['Detractors'] * summary_df_647['Weight_Company']


In [ ]:
from google.colab import files

# Save all as Excel files
summary_df_668.to_excel("summary_df_668.xlsx", index=False)
summary_df_682.to_excel("summary_df_682.xlsx", index=False)
summary_df_827.to_excel("summary_df_827.xlsx", index=False)
summary_df_647.to_excel("summary_df_647.xlsx", index=False)

# Download all files
files.download("summary_df_668.xlsx")
files.download("summary_df_682.xlsx")
files.download("summary_df_827.xlsx")
files.download("summary_df_647.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>